In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf

In [4]:
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [10]:
df=pd.read_csv("/content/evaluation.csv",sep=';',engine='python',on_bad_lines='skip')

In [11]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,1


In [12]:
df=df.dropna()

In [13]:
messages=df.copy()
y=messages['label']

In [14]:
df.head()

,Unnamed: 0,title,text,label
0,0,"Sanders back in U.S. Senate, blasts 'coloniali...",WASHINGTON (Reuters) - Democratic U.S. preside...,1.0
1,1,Kremlin: Syria peoples' congress being 'active...,MOSCOW (Reuters) - A proposal to convene a con...,1.0
2,2,Oregon Cop Convicted Of Shattering Biker’s Co...,"In a baffling fit of rage, an Oregon State Pol...",0.0
3,3,Twitter Erupts With Glee Over #CruzSexScandal...,The last thing any politician running for the ...,0.0
4,4,MUST WATCH VIDEO: Obama Tries To Trash Trump B...,This is too good to miss! Mr. Teleprompter did...,0.0


In [15]:
messages.reset_index(inplace=True)

In [16]:
df['label'].value_counts()

,count
label,
1.0,425
0.0,397


In [17]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
import re
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower().split()

  review=[ps.stem(word) for word in review if word not in stop_words]
  review=' '.join(review)
  corpus.append(review)

In [19]:
voc_size=5000
sent_length=30

In [252]:
# one_hot_rep=[one_hot(word,voc_size) for word in corpus]
# one_hot_rep
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(voc_size)
tokenizer.fit_on_texts(corpus)
sequences=tokenizer.texts_to_sequences(corpus)

In [253]:
embedded_docs=pad_sequences(sequences,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 1176,  501,  502],
       [   0,    0,    0, ...,  202,  503,  138],
       [   0,    0,    0, ..., 1179, 1180,    2],
       ...,
       [   0,    0,    0, ...,    1,  404,    2],
       [   0,    0,    0, ...,  267,  837,  106],
       [   0,    0,    0, ...,  196,  559, 2621]], dtype=int32)

In [254]:
dim=300
model=Sequential()
model.add(Embedding(voc_size,dim))
model.build((None,sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(155)))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ (None, 30, 300)        │     1,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 30, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_17                │ (None, 310)            │       565,440 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 310)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 128)            │        39,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,105,377 (8.03 MB)

 Trainable params: 2,105,377 (8.03 MB)

 Non-trainable params: 0 (0.00 B)

In [255]:
import numpy as np
y_final=np.array(y)
x_final=np.array(embedded_docs)

In [256]:
print(y[:5])  # show some labels
print(type(y))  # should be pandas.Series or list


0    1.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64
<class 'pandas.core.series.Series'>


In [257]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=98)

In [258]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=25)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.5035 - loss: 0.6871 - val_accuracy: 0.5037 - val_loss: 0.6554
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7832 - loss: 0.5355 - val_accuracy: 0.8603 - val_loss: 0.3206
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9758 - loss: 0.0891 - val_accuracy: 0.8750 - val_loss: 0.3362
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9818 - loss: 0.0427 - val_accuracy: 0.8529 - val_loss: 0.3945
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9982 - loss: 0.0087 - val_accuracy: 0.8566 - val_loss: 0.3686
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9976 - loss: 0.0036 - val_accuracy: 0.8566 - val_loss: 0.4013
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9982 - loss: 0.0031 - val_accuracy: 0.8676 - val_loss: 0.5257
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9998 - loss: 0.0011 - val_accuracy: 0.8603 - v

In [259]:
print("Type of y_train:", type(y_train))
print("First 5 values:", y_train[:5])
print("Dtype:", y_train.dtype)
print("Unique values:", np.unique(y_train))
print("Is any string in y_train?", any(isinstance(i, str) for i in y_train))
print("Shape of y_train:", y_train.shape)


Type of y_train: <class 'numpy.ndarray'>
First 5 values: [1. 0. 1. 1. 0.]
Dtype: float64
Unique values: [0. 1.]
Is any string in y_train? False
Shape of y_train: (550,)


In [260]:
ans=model.predict(x_test)
ans

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


array([[1.08236093e-02],
       [9.95853424e-01],
       [9.99911427e-01],
       [6.79338396e-01],
       [5.82405337e-05],
       [9.99883413e-01],
       [2.92208761e-01],
       [4.22983728e-02],
       [4.50451080e-06],
       [9.95761096e-01],
       [1.33433554e-03],
       [9.99775708e-01],
       [4.66907412e-01],
       [9.99157071e-01],
       [9.99880195e-01],
       [8.15124586e-02],
       [5.24945222e-02],
       [3.81197304e-01],
       [3.74806393e-03],
       [2.71366589e-05],
       [9.99251783e-01],
       [9.99927759e-01],
       [9.59779620e-01],
       [9.99223471e-01],
       [4.73998844e-05],
       [9.99950290e-01],
       [2.76183710e-02],
       [8.86115849e-01],
       [9.99437153e-01],
       [9.98703480e-01],
       [9.98172879e-01],
       [9.82601464e-01],
       [2.24409729e-01],
       [9.99014258e-01],
       [5.05302683e-04],
       [9.59975421e-01],
       [5.25606215e-01],
       [9.52149689e-01],
       [3.02686152e-04],
       [2.59788916e-03],


In [261]:
ans=np.where(ans>0.5,1,0)

In [262]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test,ans)

0.8602941176470589

In [263]:
confusion_matrix(y_test,ans)

array([[107,  15],
       [ 23, 127]])

In [264]:
print(classification_report(y_test,ans))

              precision    recall  f1-score   support

         0.0       0.82      0.88      0.85       122
         1.0       0.89      0.85      0.87       150

    accuracy                           0.86       272
   macro avg       0.86      0.86      0.86       272
weighted avg       0.86      0.86      0.86       272

